# Import necessary packages

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 3"
from kilt import retrieval
from kilt import kilt_utils as utils
import tasks
import utils
from rouge_score import rouge_scorer
import random
import numpy as np
import torch
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from datasets import load_dataset
import json
from tqdm import tqdm
import opensource

# Set up indexer

In [2]:
torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base", device_map='cuda')
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-multiset-base", device_map='cuda')
wiki = load_dataset(path='wiki_dpr', name='psgs_w100.multiset.compressed', split='train')

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenize

In [3]:
task='nq'
dataset_dpr = tasks.RQA_dpr(task=task)

In [4]:
semantic = False
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"],
                                        use_stemmer=True)
if semantic:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer
    # setup semantic model
    semantic_tokenizer = \
        AutoTokenizer.from_pretrained("microsoft/deberta-large-mnli")
    semantic_model = \
        AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-large-mnli"
        ).cuda()

# Collect data

In [5]:
indices = np.arange(len(dataset_dpr.elements))
random.shuffle(indices)
cal_indices = indices[:int(len(indices) * 0.5)]
test_indices = indices[int(len(indices) * 0.5):]

elements = dataset_dpr.elements
query = [element['question'] for element in elements]

## Setup finetuned opensource model

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
# from trl import SFTTrainer
import tasks
import utils
# from trl import DataCollatorForCompletionOnlyLM
from datasets import Dataset
import json
model_name = "NousResearch/Llama-2-7b-chat-hf"
# Fine-tuned model name
new_model = f"llama-2-7b-shuo-{task}"
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pickle
def write_list(a_list, file_name):
    # store list in binary file so 'wb' mode
    with open(file_name, 'wb') as fp:
        pickle.dump(a_list, fp)
#         print('Done writing list into a binary file')
def read_list(file_name):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

def save_results(task):
    # save retrieved_scores to a pickle file
    write_list(retrieved_scores, f'retrieved_scores_{task}.p')
    # save retrieved_true_scores to a pickle file
    write_list(retrieved_true_scores, f'retrieved_true_scores_{task}.p')
    # save queries to a pickle file
    write_list(queries, f'queries_{task}.p')
    # save answers to a pickle file
    write_list(answers, f'answers_{task}.p')
    # save passages to a pickle file
    write_list(passages, f'passages_{task}.p')
    # save opensource_true_scores to a pickle file
    write_list(opensource_true_scores, f'opensource_true_scores_{task}.p')
    # save opensource_texts to a pickle file
#     write_list(opensource_texts, f'opensource_texts_{task}.p')
    # save opensource_answers to a pickle file
    write_list(opensource_answers, f'opensource_answers_{task}.p')
    # save opensource_semantics to a picle file
    write_list(opensource_semantics, f'opensource_semantics_{task}.p')
    # save feasibilities to a pickle file
    write_list(feasibilities, f'feasibilities_{task}.p')
    # save occurances to a pickle file
    write_list(occurances, f'occurances_{task}.p')
    # save semantic_ids to a pickle file
    write_list(semantic_ids, f'semantic_ids_{task}.p')
    # save probs to a picle file
    write_list(probs, f'probs_{task}.p')

In [ ]:
def formatting_prompts_func(element):
    output_texts = []
    text = f"### Question: {element['question']}\n ### Context: {element['positive_ctxs'][0]['text']}\n ### Answer:"
    return text, element['answers'][0]

In [ ]:
element = elements[2]
prompt, answers = formatting_prompts_func(element)

In [72]:
answers

'Prize fight'

In [76]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300, return_full_text=False)
result = pipe(prompt)
print(result[0]['generated_text'])

 Prize fighter (boxing) (film) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing) (sports) (boxing)


In [74]:
new_model = f"llama-2-7b-shuo-{task}-new"
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model_new = PeftModel.from_pretrained(base_model, new_model)
# model_new = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
pipe_new = pipeline(task="text-generation", model=model_new, tokenizer=tokenizer, max_length=300, return_full_text=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

In [79]:
pipe_new = pipeline(task="text-generation", model=model_new, tokenizer=tokenizer, max_length=300, return_full_text=False)
result = pipe_new(prompt)
print(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

 Prize fight ~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!


In [80]:
answers = result[0]['generated_text']

In [60]:
answers

'Prize fight ~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!~!'

In [61]:
idx = np.char.find(answers, "~!~", start=0, end=None)

In [62]:
answers[:idx].strip()

'Prize fight'